In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import hashlib
import time
from sqlalchemy import text, inspect, MetaData, Table
from sqlalchemy.orm import sessionmaker
from app.main import getTables, Base, Account, Transaction, TransactionFinal, Load
from app.helpers.readCsv import openOneCsv, getAccountHistory, setDateColumns
from app.helpers.sqlEngine import engine

pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)


In [2]:
Session = sessionmaker(bind=engine)
session = Session()
session.connection().execute(text("DELETE FROM loads"))
session.connection().execute(text("DELETE FROM transactions_all"))
session.connection().commit()
session.close()

accounts = getTables(engine)

for index, account in accounts.iterrows():
    path = account['csvPath']
    print(path)
    
    Session = sessionmaker(bind=engine)
    session = Session()

    allFiles = getAccountHistory(path, account['id'], session)
    display(allFiles)
    allFiles.to_sql('transactions_all', con=session.connection(), if_exists='append', index=False, chunksize=10)
    session.connection().commit()
    session.close()


In [4]:
QUERY_FINAL_TRANSACTIONS = """
    SELECT
        sha256,
        postDate,
        transactionDate,
        accountId,
        amount,
        description,
        details,
        type,
        category,
        balance,
        checkOrSlipNumber,
        count(*)
    FROM 
        transactions_all
    WHERE
        postDate >= '2023-01-05'
    GROUP BY
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
    ORDER BY
        postDate DESC
"""
QUERY_FINAL_TRANSACTIONS = """
    SELECT
        sha256,
        postDate,
        transactionDate,
        accountId,
        amount,
        description,
        memo,
        details,
        type,
        category,
        balance,
        checkOrSlipNumber,
        count(*)
    FROM 
        transactions_all
    GROUP BY
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
    ORDER BY
        postDate DESC
"""
df = setDateColumns(pd.read_sql(sql=text(QUERY_FINAL_TRANSACTIONS), con=engine.connect()))
#df = setDateColumns(df)

In [11]:
QUERY_FINAL_TRANSACTIONS = """
    SELECT
        *, count(*)
    FROM 
        transactions_all
    WHERE
        description like "%BLICK%"
    GROUP BY
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15
    ORDER BY
        postDate DESC
    ;
"""
df = setDateColumns(pd.read_sql(sql=text(QUERY_FINAL_TRANSACTIONS), con=engine.connect()))
df
#df = setDateColumns(df)

In [4]:
QUERY_FINAL_TRANSACTIONS = """
    SELECT
        sha256,
        postDate,
        transactionDate,
        accountId,
        amount,
        description,
        memo,
        details,
        type,
        category,
        balance,
        checkOrSlipNumber,
        count(*)
    FROM 
        transactions_all
    WHERE
        description like "%BLICK%"
    GROUP BY
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
    ORDER BY
        postDate DESC
"""
df = setDateColumns(pd.read_sql(sql=text(QUERY_FINAL_TRANSACTIONS), con=engine.connect()))
df

In [12]:
df.dtypes

In [5]:
def describe(search, pdf):
    sdf = pdf[pdf['description'].str.contains(search, case=False)]
    display(sdf)
    amountSum = sdf['amount'].sum()
    numTransactions = sdf.shape[0]
    if numTransactions <= 0: return
    print(f"total number of transactions= {numTransactions}")
    print(f"sum= {amountSum}")
    print(f"avg cost per transaction= {round(amountSum/numTransactions, 2)}")
    latestDate = sdf.iloc[0]['postDate']
    startingDate = sdf.iloc[-1]['postDate']
    totalDays = (latestDate - startingDate).days
    print(f"dates= {startingDate} to {latestDate}, total days={totalDays}")
    avgDaysBetweenTransaction = totalDays / numTransactions
    print(f"avg days between transaction= {avgDaysBetweenTransaction}")
    amountSum
    costPerMonth = amountSum / (totalDays / 30.437)
    print(f"avg cost per month= {costPerMonth}")

In [6]:
describe('BLICK', df)

In [15]:
df.shape

In [7]:
describe('bucks', df)

In [ ]:
a="[DEBIT-2021-08-20 00:00:00-BLICK ART 800 447 1892 TAMPA FL              08/20--92.78-DEBIT_CARD-18997.7-nan]"
b="[DEBIT-2021-08-20 00:00:00-BLICK ART 800 447 1892 TAMPA FL              08/20--92.78-DEBIT_CARD-18997.70-nan]"
c="[DEBIT-2021-08-20 00:00:00-BLICK ART 800 447 1892 TAMPA FL              08/20--92.78-DEBIT_CARD-18997.7-nan]"